In [1]:
%cd ..

/Users/chanwutk/Documents/apperception


In [2]:
# Deleting the .apperception_cache if it exists, as to avoid DB conflict errors
import os
import shutil

dirpath = os.path.join('.apperception_cache')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

dirpath = os.path.join('output')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)
os.mkdir(dirpath)

# This piece of code is unsafe, and should not be run if not needed. 
# It serves for test purposes when one recieves a "dead kernel" error.
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
import os
import sys
import json
import pandas as pd
import cv2
import psycopg2
import pickle
sys.path.append(os.path.join(os.getcwd(),"apperception"))

In [4]:
from new_world import empty_world, World
from apperception.data_types import Camera
from apperception.utils import scenic_generate_df

In [5]:
World.db.reset()

In [6]:
# conn = psycopg2.connect(database="mobilitydb", user="docker", password="docker", host="localhost", port=25432)
# import_tables(conn)

In [7]:
name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")

In [8]:
# Either uncomment this cell
# df_sample_data, df_annotation = scenic_generate_df()
# with open('df_sample_data.pickle', "wb") as f:
#     pickle.dump(df_sample_data, f)
# with open('df_annotation.pickle', "wb") as f:
#     pickle.dump(df_annotation, f)

In [9]:
# Or uncomment this cell
with open('df_sample_data.pickle', "rb") as f:
    df_sample_data = pickle.loads(f.read())
with open('df_annotation.pickle', "rb") as f:
    df_annotation = pickle.loads(f.read())

In [10]:
# df_sample_data

In [11]:
# df_annotation

In [12]:
world = empty_world(name=name)

In [13]:
# cam_x, cam_y = 1317, 1463
# world.road_direction(cam_x, cam_y)

In [14]:
from apperception.utils import df_to_camera_config
# scenes = ["scene-0061", "scene-0103","scene-0553", "scene-0655", "scene-0757", "scene-0796", "scene-0916", "scene-1077", "scene-1094", "scene-1100"]
scenes = ["scene-0061"]
for scene in scenes:
    config = df_to_camera_config(scene, df_sample_data)
    camera = Camera(config=config, id=scene)
    world = world << (camera, df_annotation)
    # df_config = df_sample_data[df_sample_data['scene_name'] == scene][['sample_token']]
    # df_ann = df_annotation.join(df_config.set_index('sample_token'), on='sample_token', how='inner')
    # world = world.recognize(camera, df_annotation)

In [15]:
car_trajectories = world.get_traj()

execute: insert_cam
New camera inserted successfully.........
execute: insert_bbox_traj
Recognization done, saving to database......
execute: retrieve_traj
execute: get_traj
get_traj
done execute node


In [16]:
filtered = world.filter(lambda o, c, r: ST_X(valueAtTimestamp(o.trajCentroids, c.timestamp)) < 396.2309651419831)

In [17]:
filtered.get_traj()

execute: insert_cam
execute: insert_bbox_traj
execute: retrieve_traj
execute: filter
execute: get_traj
get_traj
done execute node


[[Trajectory(coordinates=[[386.40097211045395, 1140.689475690233, 0.0885], [386.40097211045395, 1140.689475690233, 0.0885], [386.40097211045395, 1140.689475690233, 0.1135], [386.40097211045395, 1140.689475690233, 0.1135], [386.40097211045395, 1140.689475690233, 0.1385], [386.40097211045395, 1140.689475690233, 0.1385], [386.40097211045395, 1140.689475690233, 0.1635], [386.40097211045395, 1140.689475690233, 0.1635], [386.40097211045395, 1140.689475690233, 0.1895], [386.40097211045395, 1140.689475690233, 0.1895], [386.40097211045395, 1140.689475690233, 0.2145], [386.40097211045395, 1140.689475690233, 0.2145], [386.40097211045395, 1140.689475690233, 0.2395], [386.40097211045395, 1140.689475690233, 0.2395], [386.40097211045395, 1140.689475690233, 0.2635], [386.40097211045395, 1140.689475690233, 0.2635], [386.40097211045395, 1140.689475690233, 0.2875], [386.40097211045395, 1140.689475690233, 0.2875], [386.40097211045395, 1140.689475690233, 0.3115], [386.40097211045395, 1140.689475690233, 0.3

In [18]:
(world - filtered).get_traj()

execute: insert_cam
execute: insert_bbox_traj
execute: retrieve_traj
execute: exclude
execute: insert_cam
execute: insert_bbox_traj
execute: retrieve_traj
execute: filter
done execute node
execute: get_traj
get_traj
done execute node


[[Trajectory(coordinates=[[401.34285720058557, 1193.869801800834, 0.06], [401.34285720058557, 1193.869801800834, 0.06], [402.6489291831483, 1193.6253688075658, -0.02], [402.6489291831483, 1193.6253688075658, -0.02], [404.1584778436025, 1193.5056149312122, 0.199], [404.1584778436025, 1193.5056149312122, 0.199], [405.3678572005856, 1193.362801800834, 0.298], [405.3678572005856, 1193.362801800834, 0.298], [406.56268581601023, 1193.1816800229228, 0.398], [406.56268581601023, 1193.1816800229228, 0.398], [407.4726858160102, 1193.002680022923, 0.548], [407.4726858160102, 1193.002680022923, 0.548], [408.1424938136795, 1192.9034392400918, 0.598], [408.1424938136795, 1192.9034392400918, 0.598], [409.1027733975256, 1192.6770068221751, 0.598], [409.1027733975256, 1192.6770068221751, 0.598], [409.6957733975256, 1192.5400068221752, 0.748], [409.6957733975256, 1192.5400068221752, 0.748], [410.39677339752564, 1192.3760068221752, 0.598], [410.39677339752564, 1192.3760068221752, 0.598], [411.14677339752

In [19]:
filtered.get_headings()

execute: insert_cam
execute: insert_bbox_traj
execute: retrieve_traj
execute: filter
execute: get_traj
get_traj
done execute node


[[[0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   68.702645951058,
   0.0,
   69.28582515557082,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   249.28582515557082,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   248.702645951058,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   68.9903159658931,
   0.0,
   68.9903159658299,
   0.0,
   248.96248897470127,
   0.0]],
 [[0.0,
   244.26085505185247,
   0.0,
   244.30300027267643,
   0.0,
   244.26085505185247,
   0.0,
   244.3030002726665,
   0.0,
   244.26085505185247,
   0.0,
   244.3030002726573,
   0.0,
   244.26085505187166,
   0.0,
   241.55331230336617,
   0.0,
   241.6923001327539,
   0.0,
   